In [ ]:
import os
os.chdir('/home/mmr/DUQ')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR

import numpy as np

from src.model.backbone import Backbone
from src.model.duqmodel import DUQ
from src.utils.utils import grad_penalty,train_duq,test_duq

In [ ]:
batch_size  = 100

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.286,), (0.353,))
        ])

data_root = '~/DUQ/data/fashion_mnist/'

train_dataset = torchvision.datasets.FashionMNIST(data_root,transform=transform)
train_loader  = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size)

test_dataset  = torchvision.datasets.FashionMNIST(data_root,transform=transform,train=False)
test_loader   = torch.utils.data.DataLoader(test_dataset,batch_size = batch_size)


In [ ]:
sigma = 0.1
gamma = .99
num_classes = 10
emb_size = 256
lambda_ = 0 

duq_model = DUQ(sigma,gamma,num_classes,emb_size).cuda()
opt      = optim.SGD(duq_model.parameters(),lr = 0.05,momentum=0.9,weight_decay = 1e-4)
sched    = StepLR(opt,step_size=10,gamma = 0.2)

In [ ]:
for epoch in range(30):
    train_duq(duq_model,epoch,'cuda',train_loader,opt,lambda_)
    test_duq(duq_model,epoch,batch_size,'cuda',test_loader)
    sched.step()

In [ ]:
import gc
gc.collect()

In [ ]:
# add gradient penalty  

lambda_ = 0.05 

duq_model = DUQ(sigma,gamma,num_classes,emb_size).cuda()
opt      = optim.SGD(duq_model.parameters(),lr = 0.05,momentum=0.9,weight_decay = 1e-4)
sched    = StepLR(opt,step_size=10,gamma = 0.2)

In [ ]:
for epoch in range(30):
    train_duq(duq_model,epoch,'cuda',train_loader,opt,lambda_)
    test_duq(duq_model,epoch,batch_size,'cuda',test_loader)
    sched.step()